

Prblème 1 question 3 :\
a) Estimation de la moyenne $\hat{m}$ par la méthode de Maximum de la Vraisemblance :
\
\
$\log f\left(x_{1}, x_{n} \mid m, \alpha\right)=\log  \prod_{i=1}^{n} f_{x_{i}}(x, m, \alpha)\\$
$=\sum_{i=1}^{n} \log f\left(x_{i} \mid m, \alpha\right)=\sum_{i=1}^{n} \log \left(\frac{1}{\alpha} e^{-\left(\frac{x_{i}-m}{\alpha}\right)-e^{-e}-\frac{\left(x_{i}-m\right)}{\alpha}}\right)\\$
$=\sum_{i=1}^{n}\left\{\log \left(\frac{1}{\alpha}\right)+\log \left[e^{-\left(\frac{x_{i}-m}{\alpha}\right)}\right]+\log \left[e^{-e^{-\left(\frac{x_{i}-m}{\alpha}\right)}}\right]\right\}$\
$=\sum_{i=1}^{n}\left\{\log(1)-\log (\alpha)-\left(\frac{x_{i}-m}{\alpha}\right)-e^{-\left(\frac{x_{i}-m}{\alpha}\right)}\right\}$\
$=\sum_{i=1}^{n}\left\{-\log (\alpha)-\frac{x_{i}}{\alpha}+\frac{m}{\alpha}-e^{-\left(\frac{x_{i}-m}{\alpha}\right)}\right\}$
$\left.L_{n}(\alpha, m)=n log (\alpha)-\sum_{i=1}^{n} \frac{x_{i}}{\alpha}+\frac{m m}{\alpha}-e^{\left(\frac{m}{\alpha}\right)} \sum_{i=1}^{n} e^{} \frac{\left(x_{i}\right)}{\alpha}\right)$\
$\frac{\partial}{\partial m} L_{n}(\alpha, m)=0$\
$\Rightarrow \frac{n}{\alpha}-\frac{1}{\alpha} e^{\left(\frac{\hat{m}}{\alpha}\right)} \sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\alpha}\right)}=0$\
$\Rightarrow \frac{n}{\alpha}=\frac{1}{\alpha} e^{\left(\frac{\hat{m}}{\alpha}\right)} \sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\alpha}\right)}$

$\Rightarrow \frac{n}{\sum_{i=1}^{n} e^{\left(-x_{i}\right)}}=e^{\left(\frac{\hat{m}}{\alpha}\right)} \Rightarrow \log \left(\frac{n}{\sum_{i=1}^{n} e^{\left(\frac{x_{i}}{\alpha}\right)}}\right)=\frac{\hat{m}}{\alpha}\\
\Rightarrow \hat{m}=\alpha \log \left(\frac{n}{\sum_{i=1}^{n} e\left(-\frac{x_{i}}{\alpha}\right)}\right)\\$

$\Rightarrow \hat{m}=\alpha \log \left(\frac{n}{\sum_{i=1}^{n} e\left(-\frac{x_{i}}{\alpha}\right)}\right)$

$$\Rightarrow \hat{m}=-\alpha \log \left(\frac{1}{n} \sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\alpha}\right)}\right)$$
\
\
b) Estimation de la variance $\hat{\alpha}$ par la méthode de Maximum de la Vraisemblance :


$\text { (2) } \frac{\partial}{\partial \alpha} L_{m}(\alpha, m)=\frac{\partial}{\partial \alpha}\left(-n \log (\alpha)-\sum_{i=1}^{m} \frac{x_{i}}{\alpha}+\frac{n m}{\alpha}-e^{\left(\frac{m}{\alpha}\right)} \sum_{i=1}^{m} e^{\left(-\frac{x_{i}}{\alpha}\right)}\right)=0$
\
\
On remplace m par $\hat{m}$ :

$\Rightarrow -n\hat{\alpha}+n \bar{x}+n \hat{\alpha} \log \left(\frac{\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}{n}\right)-\left[\hat{\alpha} \log \left(\frac{\sum_{i=1}^{n} e^{\left(\frac{-x_{i}}{\alpha}\right)}}{n}\right)n\left(\frac{\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}{\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}\right)\right.\\
\left.+\frac{n}{\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right).}}\left[\sum_{i=1}^{n} x_{i} e\left(\frac{-x_{i}}{\hat{\alpha}}\right)\right]\right]=0$

$\Rightarrow \alpha-\bar{x}+\frac{\sum_{i=1}^{n} x_{i}e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}{\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}=0$
\
\
En utilisant la méthode de Gauss Newton :
\
\
$\alpha_{N+1}=\alpha_{N}-\frac{f\left(\alpha_{N}\right)}{f^{\prime}\left(\alpha_{N}\right)}$


Soit $\quad f(\alpha)=\alpha-\bar{x}+\frac{\sum_{i=1}^{n} x_{i}e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}{\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\hat{\alpha}}\right)}}$
\
\
$f^{\prime}\left(\alpha\right)=1+\frac{\left[\sum_{i=1}^{n} e\left(\frac{-x_{i}}{\alpha}\right)\right]\left[\left(\frac{x_{i}}{\alpha}\right)^{2} e\left(-\frac{x_{i}}{\alpha}\right)\right]-\left[\sum_{i=1}^{n} x_{i} e^{\left(-\frac{x_{i}}{\alpha}\right)}\right]\left[\sum_{i=1}^{n} \frac {x_{i}}{\alpha^{2}} e\left(-\frac{x_{i}}{\alpha}\right)\right]}{\left[\sum_{i=1}^{n} e^{\left(-\frac{x_{i}}{\alpha}\right)}\right]^{2}}$
\
\
Etape 1 : il faut estimer ${\alpha}$ avec $\hat{\alpha}$ en utilisant Newton Raphson
On doit effectuer de 10 à 30 itérations pour trouver une solution qui converge vers $\hat{\alpha}$


Etape 2 : On utilise $\hat{\alpha}$ pour estimer $\hat{m}$


afin de determiner le moment d'ordre 1 et le moment d'ordre 2 on utilise 
la méthode MGF Momment Generating functions :

1) Momment d'ordre 1 

$
f(x)=\frac{1}{\alpha} e^{-\left(\frac{x-m}{\alpha}\right)} e^{-e^{-\left(\frac{x-m}{\alpha}\right)}}\\
\int_{-\infty}^{+\infty} f(x) d x=\int_{-\infty}^{+\infty}-e^{-y} d y \quad x, m \in \mathbb{R} ; \alpha>0\\
=\int_{0}^{\infty} e^{-y} d y=\left[-e^{-y}\right]_{0}^{\infty}=0-(-1)=1\\
\text { (*) } y=e^{-\left(\frac{x-m}{\alpha}\right)} \quad d y=\left(\frac{-1}{\alpha}\right) e^{-\left(\frac{x-m}{\alpha}\right)} d x\\
\text {  MGF : Momment Generating functions }\\
M_{x}(t)=\int_{-\infty}^{0} e^{tx} e^{-y} d y\\
\text { (*) } \Rightarrow x=m-\alpha \log (y)\\
M_{x}(t)=\int_{-\infty}^{0} e^{t\left(m-\alpha \log (y)\right)} e^{-y} d y\\
=\int_{-\infty}^{0} e^{t m} e^{-t \alpha \log (y)} e^{-y} d y\\
or \quad \Gamma: x \longmapsto \int_{0}^{+\infty} t^{x-1} e^{-t} \mathrm{~d} t\\
\Rightarrow M_{x}(t)=\left.e^{-t m}\right|_{0} ^{\infty} y^{-\alpha t} e^{-y} d y=e^{t m} \Gamma(1-\alpha t)\\
\left.E(x)=\frac{\partial M_{x}(t)}{\partial t}\right)_{t=0}=\left[m e^{t m} \Gamma(1-\alpha t)+e^{t m}((1-\alpha t)(-\alpha)\right]_{t=0}
$
$$
=m\Gamma(1) \alpha -\Gamma^{\prime}(1)=m+\alpha \gamma
$$
sachant que $\Gamma(1)=1$
avec $\gamma$: Euler-maschronic canstante $=0,577$
$$


2) moment d'ordre 2 :

$\begin{aligned}
E\left(x^{2}\right) &=\left.\frac{\partial^{2} M_{x}(t)}{\partial^{2} t}\right|_{t=0} \\
&=m\left[m e^{t m}\left[(1-\alpha t)+e^{t m} \Gamma^{\prime}(1-\alpha t)(-\alpha)\right]\right.\\
&-\alpha\left[m e^{t m} \Gamma^{\prime}(1-\alpha t)+e^{t m}\Gamma^{\prime \prime}(1-\alpha t)(-\alpha)\right|_{t=0}\\
=& m^{2}-m \alpha \Gamma^{\prime}(1)-m \alpha \Gamma^{\prime}(1)+\alpha^{2}\Gamma^{\prime \prime}(1)
\end{aligned}$
$\begin{array}{l}
\operatorname{Var}(x)=E\left(x^{2}\right)-E(x)^{2} \\
\quad=m^{2}-2 m \alpha \Gamma(1)+\alpha^{2} \Gamma^{\prime \prime}(1)-\left(m-\alpha \Gamma^{\prime \prime}(1)\right)^{2} \\
\quad=\alpha^{2} \Gamma^{\prime \prime}(1)-\alpha^{2} \Gamma^{\prime}(1)^{2} \\
\text { Soit } \\
\text { (1) } \frac{\partial \log \Gamma(x)}{\partial x}=\frac{\Gamma^{\prime}(x)}{\Gamma(x)}=\psi(x) \\
\text { (2) } \Gamma^{\prime}(x)=\Gamma(x) \psi(x) \\
\text { (3) } \Gamma^{\prime \prime}(x)=\Gamma^{\prime}(x) \psi(x)+\Gamma(x) \psi^{\prime}(x) \\
alors\\
\operatorname{Var}(x)=\alpha^{2}\left[\Gamma^{\prime}(1) \psi(1)+\Gamma(1) \cdot \psi(1)-\Gamma(1) \psi^{2}(1)\right] \\
or \quad\psi(1)=\frac{\Gamma^{\prime}(1)}{\Gamma(1)}\Rightarrow \psi(1)=\Gamma^{\prime}(1)\\ 
\Rightarrow \operatorname{Var}(x) =\alpha^{2} \psi^{\prime}(1)=\alpha^{2} \frac{\pi^{2}}{6}
\end{array}$



Methode directe de calcul du moment d'ordre 1 :

$$
\begin{array}{l}
E(x)=\int_{-\infty}^{+\infty} x f(x) d x=\int_{-\beta}^{+\infty} \frac{x}{\alpha} e^{-\left(\frac{x-m}{\alpha}\right)} e^{-\left(\frac{x-m}{\alpha}\right)} d x\\
soit:\\
y=e^{-\left(\frac{x-m}{\alpha}\right)} \quad \Rightarrow x=m-\alpha \log (y)\\
 d y=e^{-\left(\frac{x-m}{\alpha}\right)} d x\\
\Rightarrow E(x)=\int_{0}^{\infty}(m-\alpha \log (y)) e^{-y} d y\\
=m-\alpha \int_{0}^{\infty} \frac{\partial}{\partial \beta}\left[y^{\beta} e^{-y}\right]_{\beta=0}=m-\alpha \frac{\partial}{\partial \beta}\left[\int_{0}^{\infty} y^{\beta} e^{-y} d y\right]_{\beta=0}\\
=m-\alpha \frac{\partial}{\partial \beta}[\Gamma(\beta+1)]_{\beta=0}=m-\alpha \Gamma^{\prime}(\beta+1) | _{\beta=0}\\
=m-\alpha \Gamma^{\prime}(1)=m+\alpha \gamma
\end{array}
$$